In [16]:
import boto3
import pandas as pd
import numpy as np
import io
import requests
from botocore.exceptions import ClientError


In [13]:
# Set pandas to display all rows
pd.set_option('display.max_rows', None)
def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    # # When using IAM roles, boto3 retrieves credentials from the instance metadata
    # s3_client = boto3.client('s3')

    #When setting up credentials locally, use the following code
    session = boto3.Session()
    s3_client = session.client('s3')

    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_name = 'Bills/Bill_Type_Identifiers.csv'
local_file = 'Bill_Type_Identifiers.csv'

download_file_from_s3(bucket_name, s3_file_name, local_file)
download_file_from_s3(bucket_name, 'Bills/Bill_Stage_Identifiers.csv', 'Bill_Stage_Identifiers.csv')


In [6]:
df_stage = pd.read_csv('Bill_Stage_Identifiers.csv')
df_type = pd.read_csv('Bill_Type_Identifiers.csv')
print(f"{len(df_stage)} and {len(df_stage.columns)}")
print(df_stage.info())
print(df_stage.info())
print(f"{len(df_type)} and {len(df_type.columns)}")
df_stage.head()


20 and 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20 non-null     int64 
 1   name    20 non-null     object
 2   house   20 non-null     object
dtypes: int64(1), object(2)
memory usage: 608.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20 non-null     int64 
 1   name    20 non-null     object
 2   house   20 non-null     object
dtypes: int64(1), object(2)
memory usage: 608.0+ bytes
None
9 and 4


,id,name,house
0,48,Committee stage (re-committed clauses and sche...,Commons
1,49,Committee of the whole House,Commons
2,6,1st reading,Commons
3,7,2nd reading,Commons
4,28,Guillotine motion,Commons


In [8]:
df_stage = df_stage.dropna()
df_type = df_type.dropna()
df_stage = df_stage.rename(columns={'id': 'stageId'})
df_stage.head()

,stageId,name,house
0,48,Committee stage (re-committed clauses and sche...,Commons
1,49,Committee of the whole House,Commons
2,6,1st reading,Commons
3,7,2nd reading,Commons
4,28,Guillotine motion,Commons


In [15]:
df_type = df_type.rename(columns={'id': 'billTypeId'})
df_type.head()
# Set pandas to display the entire content of a column
pd.set_option('display.max_colwidth', None)

# Now display the 'description' column
print(df_type['description'])


0                                                                                                                                                                                                                                                                                                                                                                                                                   Hybrid Bills mix the characteristics of Public and Private Bills. The changes to the law proposed by a Hybrid Bill would affect the general public but would also have a significant impact for specific individuals or groups. <div><a href='https://www.parliament.uk/about/how/laws/bills/hybrid/'>Find out more about Hybrid Bills</a></div>
1                                                                                                                                                                                                                                                                  

In [25]:
df_type.info()
df_stage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   billTypeId   9 non-null      int64 
 1   category     9 non-null      object
 2   name         9 non-null      object
 3   description  9 non-null      object
dtypes: int64(1), object(3)
memory usage: 416.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stageId  20 non-null     int64 
 1   name     20 non-null     object
 2   house    20 non-null     object
dtypes: int64(1), object(2)
memory usage: 608.0+ bytes


In [24]:
def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
upload_df_to_s3(df_stage, bucket_name, 'Preprocessing/Bills/Bill_Stage_Identifiers.csv')
upload_df_to_s3(df_type, bucket_name, 'Preprocessing/Bills/Bill_Type_Identifiers.csv')


True